![logo](./img/LogoLine_horizon_C3S.png)

<br>

# Merge Reanalysys (ERA5) and Hindcast (SEAS5) for modelling

### About


The notebook has the following outline:
* 1 - Join hindcast files: tp and t2m for each month, year and member
* 2 - Merge ERA5 with hindcast files (ERA5: warm up period=5 years)
* 3 - Combine datasets of subcatchments (C1 and C2) for modelling (PERSiST)

### Install packages

In [1]:
# Install CDS API for downloading data from the CDS
#!pip install cdsapi

In [ ]:
# Install cfgrib to enable us to read GRIB format files
#!conda install -c conda-forge cfgrib -y

In [1]:
# Miscellaneous operating system interfaces
import os

# CDS API
import cdsapi

# To map GRIB files to NetCDF Common Data Model
import cfgrib

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd
import scipy

# Libraries for plotting and geospatial data visualisation
import matplotlib.path as mpath
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

# To work with data labels in dictionary format
from collections import OrderedDict

# Date and time related libraries
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import datetime

# Interactive HTML widgets
import ipywidgets as widgets

# Disable warnings for data download via API
import urllib3 
urllib3.disable_warnings()

Here we specify a data directory where the hindcast files are located and where we will save generated files

In [19]:
input_path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast'
output_path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast_joined'

# 1. Join precipitation and temperature hindcast files

In [24]:
# Function to generate temperature filenames
def generate_temperature_filename(member, year, month):
    return f"hind_C2_t2m_member{member}_year{year}_month{month}.csv"

# Function to generate precipitation filenames
def generate_precipitation_filename(member, time):
    return f"hind_C2_tp_member{member}_time{time}.csv"

# Time indices for precipitation files (0 to 287)
precipitation_times = range(288)

# Loop through each member
for member in range(25):
    # Counters to track the current year and month for temperature files
    current_year = 0
    current_month = 0
    
    for time in precipitation_times:
        # Generate filenames
        temperature_filename = generate_temperature_filename(member, current_year, current_month)
        precipitation_filename = generate_precipitation_filename(member, time)
        
        # Construct full file paths
        temperature_filepath = os.path.join(input_path, temperature_filename)
        precipitation_filepath = os.path.join(input_path, precipitation_filename)
        
        # Check if files exist before processing
        if not os.path.exists(temperature_filepath) or not os.path.exists(precipitation_filepath):
            print(f"Skipping non-existing file pair: {temperature_filename} and {precipitation_filename}")
            continue
        
        # Read the temperature and precipitation CSV files
        temperature_df = pd.read_csv(temperature_filepath, header=None, names=['Temperature'])
        precipitation_df = pd.read_csv(precipitation_filepath, header=None, names=['Precipitation'])
        
        # Merge the data
        merged_df = pd.concat([precipitation_df, temperature_df], axis=1)
        
        # Save the merged data to a new CSV file
        output_filename = f"hind_C2_joined_member{member}_year{current_year}_month{current_month}.csv"  
        merged_df.to_csv(os.path.join(output_path, output_filename), sep=' ', index=False)
        
        # Update the year and month counters
        current_month += 1
        if current_month > 11:
            current_month = 0
            current_year += 1

# 2 - Merge ERA5 with hindcast files (ERA5: warm up period=5 years)

In [2]:
#Set path directories
path_reanalysis = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/reanalysis'
path_hindcast = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast_joined'
path_merged = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast_merged'

In [66]:
#TEST TO MERGE ERA5 AND HINDCAST

# Load reanalysis data
df = pd.read_csv(f'{path_reanalysis}/reanalysis_daily_C1_all_withDates.csv')

# Split 'time tp t2m' column into separate columns
df[['time', 'tp', 't2m']] = df['time tp t2m'].str.split(expand=True)

# Drop the original 'time tp t2m' column
df.drop(columns=['time tp t2m'], inplace=True)

# Define the start and end dates for the 5-year period
start_date = '2019-01-01'
end_date = '2024-01-01'

# Filter the DataFrame based on the specified date range
df_5_years = df[(df['time'] >= start_date) & (df['time'] < end_date)]

# Select meteo data only without dates
df_era5 = df_5_years[['tp', 't2m']]

# Load hindcast data
df_hind = pd.read_csv(f'{path_hindcast}/hind_C1_joined_member0_year0_month0.csv')

# Split 'Precipitation Temperature' column into separate columns
df_hind[['tp', 't2m']] = df_hind['Precipitation Temperature'].str.split(expand=True)

# Drop the original 'Precipitation Temperature' column
df_hind.drop(columns=['Precipitation Temperature'], inplace=True)

# Merge reanalysis and hindcast data
df_merged = pd.concat([df_era5, df_hind], ignore_index=True)

In [67]:
df_merged

,tp,t2m
0,0.0,3.1316223
1,0.0,3.1357422
2,0.0,1.1793518
3,0.0,0.42651367
4,0.0,2.979004
...,...,...
2036,0.0,20.076019
2037,0.0,19.072723
2038,0.0,19.254303
2039,0.0,19.23053


In [76]:
#TEST TO EXTRACT DATE FROM HINDCAST FILES

import glob

def extract_date(year_numeric, month_numeric):
    # Map numeric year to actual year
    year = 1993 + year_numeric
    
    # Map numeric month to actual month
    month_mapping = {
        0: 'January', 1: 'February', 2: 'March', 3: 'April',
        4: 'May', 5: 'June', 6: 'July', 7: 'August',
        8: 'September', 9: 'October', 10: 'November', 11: 'December'
    }
    month = month_mapping[month_numeric]
    
    # Convert year and month to date without time
    date = pd.to_datetime(str(year) + '-' + str(month_numeric + 1), format='%Y-%m').date()
    
    return date

# Find all CSV files matching the pattern
csv_files = glob.glob(path_hindcast + '/*.csv')

for csv_file in csv_files:
    # Extract year and month from file name
    file_parts = csv_file.split('_')
    member = int(file_parts[-3][6:])  # Extract member number from the third to last part
    year_numeric = int(file_parts[-2][4:])  # Extract numeric year from the second to last part
    month_numeric = int(file_parts[-1].split('.')[0][5:])  # Extract numeric month from the last part
    
    # Extract date using the function
    date = extract_date(year_numeric, month_numeric)
    
    # Display the details
    #print(f"CSV File: {csv_file}, Member: {member}, Date: {date}")

Final code to merge datasets and create csv files

In [5]:
reach='C2' #Subcatchments, reach1=C1, reach2=C2

In [6]:
def extract_date(year_numeric, month_numeric):
    # Map numeric year to actual year
    year = 1993 + year_numeric
    
    # Convert to date object
    return pd.to_datetime(f'{year}-{month_numeric + 1}', format='%Y-%m').date()

def get_reanalysis_data_for_period(reanalysis_df, end_date):
    # Convert end_date to datetime
    end_date = pd.to_datetime(end_date)
    start_date = end_date - pd.DateOffset(years=5)
    
    # Filter the reanalysis DataFrame based on the specified date range
    df_5_years = reanalysis_df[(reanalysis_df['time'] >= start_date.strftime('%Y-%m-%d')) & 
                               (reanalysis_df['time'] < end_date.strftime('%Y-%m-%d'))]
    
    # Select meteo data only without dates
    df_era5 = df_5_years[['tp', 't2m']]
    return df_era5

# Load reanalysis data
df_reanalysis = pd.read_csv(f'{path_reanalysis}/reanalysis_daily_{reach}_all_withDates.csv')

# Split 'time tp t2m' column into separate columns
df_reanalysis[['time', 'tp', 't2m']] = df_reanalysis['time tp t2m'].str.split(expand=True)

# Drop the original 'time tp t2m' column
df_reanalysis.drop(columns=['time tp t2m'], inplace=True)

# Specify the ranges for member, year, and month
members = range(25)  # 0 to 24
years = range(24)  # 0 to 23
months = range(12)  # 0 to 11

for member in members:
    for year in years:
        for month in months:
            # Construct the filename
            csv_file = f"{path_hindcast}/hind_{reach}_joined_member{member}_year{year}_month{month}.csv"
            
            # Check if the file exists
            if os.path.exists(csv_file):
                # Extract date using the function
                date = extract_date(year, month)
                
                # Get reanalysis data for the 5 years before the extracted date
                df_era5 = get_reanalysis_data_for_period(df_reanalysis, date)
                
                # Load hindcast data
                df_hind = pd.read_csv(csv_file)
                
                # Split 'Precipitation Temperature' column into separate columns
                df_hind[['tp', 't2m']] = df_hind['Precipitation Temperature'].str.split(expand=True)
                
                # Drop the original 'Precipitation Temperature' column
                df_hind.drop(columns=['Precipitation Temperature'], inplace=True)
                
                # Merge reanalysis and hindcast data
                df_merged = pd.concat([df_era5, df_hind], ignore_index=True)
                
                # Save the merged DataFrame to a new CSV file with a space separator
                output_file = os.path.join(path_merged, f"hind_{reach}_merged_member{member}_year{year}_month{month}_date{date}.csv")
                df_merged.to_csv(output_file, index=False)
                
print("Processing complete.")

Processing complete.


# 3 - Combine datasets of subcatchments (C1 and C2) for modelling (PERSiST)

In [ ]:
input_path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast_merged'
output_path = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/seasonal/hindcast_modelling'

In [ ]:
import glob

# Find all CSV files in the input directory
csv_files = glob.glob(os.path.join(input_path, '*.csv'))

# Create a dictionary to hold pairs of files
file_pairs = {}

# Identify and pair up files with 'C1' and 'C2' in the filename
for file in csv_files:
    filename = os.path.basename(file)
    if 'C1' in filename:
        key = filename.replace('C1', '')
        if key not in file_pairs:
            file_pairs[key] = {}
        file_pairs[key]['C1'] = file
    elif 'C2' in filename:
        key = filename.replace('C2', '')
        if key not in file_pairs:
            file_pairs[key] = {}
        file_pairs[key]['C2'] = file

# Process each pair of files
for key, pair in file_pairs.items():
    if 'C1' in pair and 'C2' in pair:
        # Load the CSV files into pandas DataFrames
        df1 = pd.read_csv(pair['C1'])
        df2 = pd.read_csv(pair['C2'])

        # Get the number of rows in df1
        num_rows_df1 = len(df1)

        # Create new rows without decimals
        new_row_df1 = pd.DataFrame({'tp': [str(num_rows_df1)], 't2m': [None]})
        new_row_2 = pd.DataFrame({'tp': [str(2)], 't2m': [None]})
        new_row_3 = pd.DataFrame({'tp': ['C1'], 't2m': [None]})

        # Add the rows in the correct order to df1
        df1 = pd.concat([new_row_df1, new_row_2, new_row_3, df1]).reset_index(drop=True)

        # Add the name 'C2' as a new row at index 0 in df2
        new_row_4 = pd.DataFrame({'tp': ['C2'], 't2m': [None]})
        df2 = pd.concat([new_row_4, df2], ignore_index=True)

        # Concatenate the DataFrames
        result = pd.concat([df1, df2], ignore_index=True)

        # Write the result to a new CSV file without column headers and with space separator
        output_filename = f'{key}.csv'
        result.to_csv(os.path.join(output_path, output_filename), index=False, header=False, sep=' ')